# 3. Image Classification

We import everything that we will need, and instantiate a device that we want to use.

In [1]:
# import everything
import torch
import torchvision
device = "cuda" if torch.cuda.is_available() else "cpu"

### 3. (c) Network Implementation

Implement and instantiate the network as given. Since there are various different ways to implement this network, no guidelines will be provided here.

In [2]:
# implement and instantiate a network
def convolutional(Q1, Q2, O):
  return torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=3, out_channels=Q1, kernel_size = (5,5), stride=1, padding=2),
    torch.nn.MaxPool2d(kernel_size=(2,2), stride=2),
    torch.nn.Tanh(),
    torch.nn.Conv2d(in_channels=Q1, out_channels=Q2, kernel_size = (5,5), stride=1, padding=2),
    torch.nn.MaxPool2d(kernel_size=(2,2), stride=2),
    torch.nn.Tanh(),
    torch.nn.Flatten(),
    torch.nn.Linear(Q2*8*8, O)
  )

network = convolutional(16, 32, 10)

### 3. (d) Dataset and Data Loaders

We make use of the CIFAR-10 dataset, which is part of the `torchvision.datasets`. Instantiate training and test data loaders for this dataset.

In [3]:
# instantiate training data loader
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# instantiate test data loader
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:06<00:00, 27048979.90it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### 3. (e) Network Training

Train the network using appropriate optimizer and loss function for 10 epochs. Compute validation set accuracy at the end of each epoch.

In [4]:
# instantiate optimizer and loss
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=network.parameters(),lr=0.01, momentum=0.8) #could also use Adam optimiser 

test_acc = []
network = network.to(device)

# train for 10 epochs
for epoch in range(10):
  # train the network on the batches of the training set
  for x,t in train_loader: 
    optimizer.zero_grad()
    x, t = x.to(device), t.to(device)

    y = network(x)
    l = loss(y, t)
    l.backward()
    optimizer.step()

  # compute test set accuracy
  with torch.no_grad():
      for x, t in test_loader:
        x, t = x.to(device), t.to(device)
        y = network(x)
        test_acc.append((y.argmax(dim=1) == t).float().mean().item())

  # report test set accuracy
  print(f"Epoch {epoch+1}: test set accuracy: {sum(test_acc)/len(test_acc)}")

Epoch 1: test set accuracy: 0.4721337579617834
Epoch 2: test set accuracy: 0.47636345541401276
Epoch 3: test set accuracy: 0.5090233545647559


KeyboardInterrupt: 